In [13]:
import pandas as pd
import os
from env import get_db_url
import matplotlib as plt
import seaborn as sns
import numpy as np

### Acquire and Summarize

Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [ ]:
sql = '''SELECT *
FROM properties_2017
LEFT JOIN predictions_2017 USING (parcelid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
LEFT JOIN storytype USING (storytypeid)
LEFT JOIN unique_properties USING (parcelid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
WHERE propertylandusetype.propertylandusedesc = 'Single Family Residential'
AND predictions_2017.transactiondate LIKE '2017%%';'''

df = pd.read_sql(sql, get_db_url('zillow'));

In [ ]:
df.head()

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df_num = df.select_dtypes(include='number')
df_num

In [ ]:
# value counts for each column 
for column in df_num.columns:
    print(column)
    print(df_num[column].value_counts(ascending=False))
    print("-----------------")

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def find_missing_values(df):
    column_name = []
    num_rows_missing = []
    pct_rows_missing = []

    for column in df.columns:       
        num_rows_missing.append(df[column].isna().sum())
        pct_rows_missing.append(df[column].isna().sum()/ len(df))
        column_name.append(column)
    data = {'column_name':column_name, 'num_rows_missing': num_rows_missing, 'pct_rows_missing': pct_rows_missing}
    return pd.DataFrame(data, index=None)


In [ ]:
# call function and show dataframe
missing_values = find_missing_values(df)
missing_values.head()

In [ ]:
# set index to column name so that erroneous value is not assigned to index
missing_values.set_index('column_name', inplace=True)

In [ ]:
missing_values.head()

### Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.